# Problem 2

## Spectral graph teory: cuts, communities and conductance

In [1]:
import numpy as np
import networkx as nx 

from itertools import combinations
from tqdm import tqdm

### (1) Find the densest subgraph using the Charikar greedy algorithm

In [2]:
nodes = list(range(1,11))

edges = [
    (1,2),
    (1,3),
    (1,4),
    (1,7),
    (2,3),
    (2,4),
    (2,5),
    (3,4),
    (3,7),
    (4,5),
    (5,6),
    (5,9),
    (6,8),
    (6,9),
    (6,10),
    (7,8),
    (7,10),
    (8,9),
    (8,10),
    (9,10)
]

In [3]:
# Adjacency matrix

A = np.zeros((10,10))

for e in edges:
    A[e[0] - 1, e[1] - 1] = 1
    A[e[1] - 1, e[0] - 1] = 1

# Degree matrix

D = np.diag(np.sum(A, axis = 0))

# Laplacian matrix

L = D - A

# Adjacency lists

adj_lists = {v:[] for v in nodes}

for e in edges:
    adj_lists[e[0]].append(e[1])
    adj_lists[e[1]].append(e[0])

In [4]:
adj_lists

{1: [2, 3, 4, 7],
 2: [1, 3, 4, 5],
 3: [1, 2, 4, 7],
 4: [1, 2, 3, 5],
 5: [2, 4, 6, 9],
 6: [5, 8, 9, 10],
 7: [1, 3, 8, 10],
 8: [6, 7, 9, 10],
 9: [5, 6, 8, 10],
 10: [6, 7, 8, 9]}

In [5]:
def Greedy_Densest_Subgraph(
        adj_lists:dict
        ) -> list:
    
    S = list(adj_lists.keys())

    S_G = list(adj_lists.keys())

    f_SG = 0.5*sum(map(lambda x: len(adj_lists[x]), adj_lists))/len(S)

    print(f_SG)
    
    while len(S) > 1:

        v = min(adj_lists, key = lambda k: len(adj_lists[k]))

        S.remove(v)

        adj_lists = {u : [node for node in adj_lists[u] if node != v] for u in list(adj_lists.keys()) if u != v}

        print(adj_lists)
        
        f_S = 0.5*sum(map(lambda x: len(adj_lists[x]), adj_lists))/len(S)
        
        print(f_S)
        
        if f_S >= f_SG:
               
               f_SG = f_S

               S_G = S

    return S_G

In [6]:
Greedy_Densest_Subgraph(adj_lists)

2.0
{2: [3, 4, 5], 3: [2, 4, 7], 4: [2, 3, 5], 5: [2, 4, 6, 9], 6: [5, 8, 9, 10], 7: [3, 8, 10], 8: [6, 7, 9, 10], 9: [5, 6, 8, 10], 10: [6, 7, 8, 9]}
1.7777777777777777
{3: [4, 7], 4: [3, 5], 5: [4, 6, 9], 6: [5, 8, 9, 10], 7: [3, 8, 10], 8: [6, 7, 9, 10], 9: [5, 6, 8, 10], 10: [6, 7, 8, 9]}
1.625
{4: [5], 5: [4, 6, 9], 6: [5, 8, 9, 10], 7: [8, 10], 8: [6, 7, 9, 10], 9: [5, 6, 8, 10], 10: [6, 7, 8, 9]}
1.5714285714285714
{5: [6, 9], 6: [5, 8, 9, 10], 7: [8, 10], 8: [6, 7, 9, 10], 9: [5, 6, 8, 10], 10: [6, 7, 8, 9]}
1.6666666666666667
{6: [8, 9, 10], 7: [8, 10], 8: [6, 7, 9, 10], 9: [6, 8, 10], 10: [6, 7, 8, 9]}
1.6
{6: [8, 9, 10], 8: [6, 9, 10], 9: [6, 8, 10], 10: [6, 8, 9]}
1.5
{8: [9, 10], 9: [8, 10], 10: [8, 9]}
1.0
{9: [10], 10: [9]}
0.5
{10: []}
0.0


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

### (3) Show that Cheeger's inequality hold for this graph

Given the second smallest eigenvalue $\lambda_2$ of the graph laplacian, Cheeger's inequality states that the graph conductance $\phi_G$ is bounded as follows:

$$ \frac{\lambda_2}{2} \leq \phi_G \leq \lambda_2 $$

where the conductance is defined as the minimum among the conductance of all cuts, where a cut conductance is the cardinality of the cut over its volume as the minimum between the sum of the degrees of the nodes withing one of the two partitions:

$$ \phi_G = \underset{C1 \subset V, C2 = V \setminus C1}{\mathrm{min}} \ \ \frac{|E(C1,C2)|}{\min\{ \sum_{i \in C1} d_i, \sum_{i \in C2} d_i\}} $$

Since the given graph is quite small, we can compute its conductance explicitely:

In [7]:
cuts_conductance = {}

for m in tqdm(range(1,6)):

    cuts = list(combinations(nodes, m))

    for cut in cuts:

        C1 = cut

        C2 = [node for node in nodes if node not in cut]

        E_C1C2 = len([e for e in edges if (e[0] in C1 and e[1] in C2) or (e[1] in C1 and e[0] in C2)])

        sum_d1 = sum(map(len, 
                         dict(
                             filter(
                                 lambda item: item[0] not in C1, adj_lists.items())).values()))
        
        sum_d2 = sum(map(len, 
                         dict(
                             filter(
                                 lambda item: item[0] not in C2, adj_lists.items())).values()))
        

        cuts_conductance[C1] = E_C1C2 / min((sum_d1, sum_d2))
                

100%|██████████| 5/5 [00:00<00:00, 833.63it/s]


In [9]:
min_cut, min_phi_G = min(cuts_conductance.items(), key = lambda x: x[1])

In [10]:
min_cut

(1, 2, 3, 4, 5)

In [11]:
min_phi_G

0.2

Now to check if Cheeger's inequality holds we need the second smallest eigenvalue of the normalized laplacian matrix: since our graph is $d$-regular, with $d = 4$ we have that 
$$ \mathcal{L} = \mathbb{I} - \frac{1}{d}A $$

In [12]:
# Normalized laplacian of a d-regular graph with d = 4

d = 4
L_ = np.eye(L.shape[0]) - A/d

# Eigendecomposition through numpy functions

lambdas, U = np.linalg.eig(L_)

In [13]:
# We call np.real over the eigenvalues to get rid of numerical problems with imaginary parts

lambda_2 = np.sort(np.real(lambdas))[1]

In [14]:
lambda_2

0.2500000000000001

In [15]:
(lambda_2/2 <= min_phi_G) & (min_phi_G <= np.sqrt(2*lambda_2))

True

### (4) Find the cut that satisfies Cheeger's inequality

We are given a $d$-regular graph ($d = 4$), so we can use the ```SWEEPING``` algorithm we have seen during lectures.

This algorithm leverages the eigendecomposition of the normalized laplacian of an undirected $d$-regular graph $G(V,E)$, giving as output a cut $(C_1, C_2)$ minimizing the conductance. 

First of all we retrieve the eigendecomposition of the normalized laplacian for the graph, being as we said before
$$ \mathcal{L} = \mathbb{I} - \frac{1}{d}A $$

and we collect the eigenvector $v_2$ corresponding to the second smallest eigenvalue: according to ```numpy``` documentation, eigenvalues retrieved through ```linalg.eig``` are not necessarily sorted, so we have to sort them as well as we have to sort accordingly the eigenvectors, which are given in a matrix of column vectors. 

In [16]:
# Just repeating previous computations

d = 4
L_ = np.eye(L.shape[0]) - A/d

lambdas, U = np.linalg.eig(L_)

In [17]:
# We have to sort properly both the eigenvalues and the eigenvectors

idxs = np.argsort(np.real(lambdas))
lambdas = np.real(lambdas)[idxs]
U = np.real(U[:,idxs])

In [18]:
v_2 = U[:,1]

Now let's sort the nodes increasingly with respect to $v_2$:

In [19]:
nodes_sorted = np.argsort(v_2)

In [20]:
nodes_sorted

array([7, 8, 5, 9, 6, 4, 0, 2, 1, 3], dtype=int64)

In [21]:
v_2

array([ 3.53553391e-01,  3.53553391e-01,  3.53553391e-01,  3.53553391e-01,
        9.29592289e-18, -3.53553391e-01, -4.44460925e-17, -3.53553391e-01,
       -3.53553391e-01, -3.53553391e-01])

The next step is a loop over all the vertices sorted with respect to $v_2$. We consider the sorted vector to be $x_1 \leq x_2 \leq ... \leq x_n$, so that the index point to the respective vertex. 

Now we perform the "sweeping" step: we iterate over $i \in \{1,2,..,n -1\}$ and retrieve the conductance associated to all the cuts $C_1 = \{1,...,i\}, C_2 = V \backslash C_1$: then we retrieve the cut whose conductance is the minimum. 

In [22]:
phis = {}
C1 = []
C2 = nodes

for i in range(len(nodes)-1):
    
    # Do the sweeps between the two cuts

    C1.append(nodes_sorted[i]+1)
    C2.remove(nodes_sorted[i]+1)
    
    # As before collect the quantities need to compute conductance of the cut of interest
    
    E_C1C2 = len([e for e in edges if (e[0] in C1 and e[1] in C2) or (e[1] in C1 and e[0] in C2)])

    sum_d1 = sum(map(len, 
                        dict(
                            filter(
                                lambda item: item[0] not in C1, adj_lists.items())).values()))
    
    sum_d2 = sum(map(len, 
                        dict(
                            filter(
                                lambda item: item[0] not in C2, adj_lists.items())).values()))
    

    phis[tuple(C1)] = E_C1C2 / min((sum_d1, sum_d2))

In [23]:
phis

{(8,): 1.0,
 (8, 9): 0.75,
 (8, 9, 6): 0.5,
 (8, 9, 6, 10): 0.25,
 (8, 9, 6, 10, 7): 0.2,
 (8, 9, 6, 10, 7, 5): 0.25,
 (8, 9, 6, 10, 7, 5, 1): 0.5,
 (8, 9, 6, 10, 7, 5, 1, 3): 0.75,
 (8, 9, 6, 10, 7, 5, 1, 3, 2): 1.0}

In [24]:
min_cut_SW, min_phi_G_SW = min(phis.items(), key = lambda x: x[1])

In [25]:
min_cut_SW

(8, 9, 6, 10, 7)

In [26]:
min_phi_G_SW

0.2

We found the exactly same cut: we just got in output the complementary set of the one retrieved with the brutal enumeration. This algorihtm yields a good approximation for the computation of the conductance, providing an heuristic for a NP-hard problem that drops drastically the computation complexity. 

__________